# Comparison for Dirichlet and Neumann BCs for simple plane wave problem.

In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
from netgen.webgui import Draw as Drawgeo
import matplotlib.pyplot as plt
import numpy as np


importing NGSolve-6.2.2301


In [2]:
rect=WorkPlane().RectangleC(5,5).Face()

geo = OCCGeometry(rect)
mesh = Mesh(geo.GenerateMesh (maxh=0.3))

## Defining Plane Wave

In [3]:
K = CF((0, 1, 0))
P = CF((1,0,0))
phasor = exp(1j * ((K[0] * x) + (K[1] * y) + (K[2]*z)))

ex = P[0] * phasor
ey = P[1] * phasor
ez = P[2] * phasor

e = CF((ex, ey, ez))

Draw(e, mesh)

def do_curl(e):
    e1 = e[0]
    e2 = e[1]
    e3 = e[2]

    c = CF((e3.Diff(y) - e2.Diff(z), e3.Diff(x) - e1.Diff(z), e2.Diff(x) - e1.Diff(y)))
    return c

curle = do_curl(e)


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

## Using Dirichlet BCs

In [4]:
fes = HCurl(mesh, order=4, complex=True, dirichlet="default")
u = fes.TrialFunction()
v = fes.TestFunction()

omega = sqrt(K[0]**2 + K[1]**2 + K[2]**2)

a = BilinearForm(fes, symmetric=True)
a += curl(u)*curl(v)*dx  - omega**2*u*v*dx
a.Assemble()

b = LinearForm(fes)
b.Assemble()

gfu = GridFunction(fes)
gfu.Set((e), BND)

r = b.vec - a.mat * gfu.vec
gfu.vec.data += a.mat.Inverse(freedofs=fes.FreeDofs()) * r
Draw(gfu[0], mesh)
#Draw(gfu[1], mesh)
#Draw(gfu[2], mesh)

used dof inconsistency
(silence this warning by setting BilinearForm(...check_unused=False) )


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

## Using Neumann BCs:

In [5]:
fes = HCurl(mesh, order=4, complex=True)
u = fes.TrialFunction()
v = fes.TestFunction()

omega = sqrt(K[0]**2 + K[1]**2 + K[2]**2)

a = BilinearForm(fes, symmetric=True)
a += curl(u)*curl(v)*dx  - omega**2*u*v*dx
a.Assemble()

n = specialcf.normal(3)
n_curle = Cross(n, curle)
nn_curle = Cross(n, n_curle)

b = LinearForm(fes)
b +=  nn_curle * v.Trace() * ds(definedon='default')
b.Assemble()

gfu = GridFunction(fes)

gfu.vec.data += a.mat.Inverse(freedofs=fes.FreeDofs()) * b.vec
Draw(gfu[0].real, mesh)
# Draw(gfu[1], mesh)
# Draw(gfu[2], mesh)

used dof inconsistency
(silence this warning by setting BilinearForm(...check_unused=False) )


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene